In [1]:
# 处理情感词词典
from __future__ import division
import pandas

class SentiWordNet():
    def __init__(self,netpath):
        self.netpath = netpath
        self.dictionary = {}
    
    def infoextract(self):
        tempdict = {}
        templist = []
        try:
            f = open(self.netpath,"r")
        except IOError:
            print("failed to open file!")
            exit()
        print('start extracting.......')
        
    # Example line:
    # POS     ID     PosS  NegS SynsetTerm#sensenumber Desc
    # a   00009618  0.5    0.25  spartan#4 austere#3 ascetical#2  ……

    
        cnt = 0
        for sor in f.readlines():
            cnt += 1
            if(cnt%1000==0):
                print(cnt*100/117687, '%')
            if sor.strip().startswith("#"):
                pass
            else:
                data = sor.split("\t")
                if len(data) != 6:
                    print('invalid data')
                    break
                    
                wordTypeMarker = data[0]      
                synsetScore = float(data[2]) - float(data[3])   #// Calculate synset score as score = PosS - NegS
                synTermsSplit = data[4].split(" ")    # word#sentimentscore
                
                for w in synTermsSplit:
                    synTermAndRank = w.split("#")           #
                    synTerm = synTermAndRank[0] + "#" + wordTypeMarker    #单词#词性
                    synTermRank = int(synTermAndRank[1])    
                    if  synTerm in tempdict:
                        t = [synTermRank,synsetScore]
                        tempdict.get(synTerm).append(t)            
                    else:
                        temp = {synTerm:[]}
                        t = [synTermRank,synsetScore]
                        temp.get(synTerm).append(t)
                        tempdict.update(temp) 
#                         print(tempdict)

                for key in tempdict.keys():
                    score = 0.0
                    ssum = 0.0            
                    for wordlist in tempdict.get(key):
#                         print(wordlist)
                        score += wordlist[1]/wordlist[0]
                        ssum += 1.0/wordlist[0]
                        score /= ssum
                        self.dictionary.update({key:score})
    
    def getscore(self,word,pos):
        return self.dictionary.get(word + "#" + pos)
             
            
                




In [2]:
import os
import pandas as pd

if __name__ == '__main__':
    os.chdir(os.getcwd())
    if os.path.exists('./dictionaries/sentiment_dictionary/SentiWordNet3.0.0.csv'):
        swn_data = pd.read_csv('./dictionaries/sentiment_dictionary/SentiWordNet3.0.0.csv')
        swn_data.head()
        
    else:
        
        netpath = r'./dictionaries/sentiment_dictionary/SentiWordNet_3.0.0.txt'
        swn= SentiWordNet(netpath)
        swn.infoextract()
        
        word = []
        attr = []
        score = []
        for key in swn.dictionary:
            word.append(key.split('#')[0])
            attr.append(key.split('#')[1])
            score.append(swn.dictionary[key])

        dataframe = pd.DataFrame({'word':word, 'attr':attr, 'score':score})
        dataframe.to_csv("swn.csv",index=False,sep=',')
    

In [3]:
swn_data.head()

,word,attr,score
0,able,a,0.079091
1,unable,a,-0.363636
2,dorsal,a,0.000000
3,abaxial,a,0.000000
4,ventral,a,0.000000


In [59]:
# 处理程度副词词典
def get_degree_dictionary(path):
    
    groups = ['1', '2', '3', '4', '5']
    
    with open(path, 'r') as f:
        words = []
        weights = []
        weight = 1
        for line in f.readlines():
            if line[0] == '#' or line.strip() == '':
                continue
#             print(line[0])
            
            if line[0] in groups:
#                 print(line[0])
                weight = 2 - (int(line[0])-1)*0.3
                continue
            
            words.append(line.strip('\n'))
            weights.append(weight)
        
    degree_pd = pandas.DataFrame({'degree_word': words, 'weight': weights})
    
    print(degree_pd)
    
    degree_pd.to_csv('./dictionaries/degree_dictionary/degree_dictionary.csv',
                     index=False)
        
        
        
        
    
    

In [60]:
get_degree_dictionary(path='./dictionaries/degree_dictionary/degree_dictionary.txt')

       degree_word  weight
0         absolute     2.0
1       absolutely     2.0
2       alarmingly     2.0
3        amazingly     2.0
4    astonishingly     2.0
..             ...     ...
102          light     0.8
103         merely     0.8
104       relative     0.8
105         slight     0.8
106       slightly     0.8

[107 rows x 2 columns]


In [61]:
def get_negative_dictionary(path):
    negative_pd = pd.DataFrame()
    words = []
    
    with open(path, 'r') as f:
        for line in f.readlines():
            words.append(line.strip('\n'))
    
    negative_pd['word'] = words
    
    negative_pd.to_csv('./dictionaries/negative_dictionary/negative_dictionary.csv',index=False)
    
    

In [62]:
get_negative_dictionary(path='./dictionaries/negative_dictionary/negative_dictionary.txt')